In [126]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException
import json, os, subprocess, re, pprint


# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文


# 使用 Chrome 的 WebDriver
my_service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(
    options = my_options,
    service = my_service
)

folderPath = 'Gutenberg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
listData = []
list_Data = []


In [127]:
url = "https://www.gutenberg.org/browse/languages/zh"

def visit():
    driver.get(url)
    
def getLinks():
    
    linkElement = driver.find_elements(By.CSS_SELECTOR, 'ul>li.pgdbetext>a')
    
    # 將任何以非文字字元、英文為title的書轉為空字串
    for a in linkElement:
        regex = r'[\W]+' 
        regex1 = r'[a-zA-Z]+' 
        title = a.get_attribute('innerText')
        title = re.sub(regex, '',title)
        title = re.sub(regex1, '',title)
        
        #加入列表
        listData.append({
            'link' : a.get_attribute('href'),
            'title' : title  
        })
    
    # 從列表篩選出title為中文的書
    for item in listData:
        title = item['title']
        if re.search(r'[\u4e00-\u9fff]+', title):  # 判斷 title 是否含有中文字符
            list_Data.append(item)
    
#     pprint.pprint(list_Data)

# 進入每本書的link，並點選Plain Text UTF-8抓取內文
def getSublinks():
    count = 0
    regex = r'[\u4e00-\u9fff．，,。、；：！？?《》『』「」]+'
    for item in list_Data:
        driver.get(item['link'])
        driver.find_element(By.LINK_TEXT, "Plain Text UTF-8").click()
        
        with open(f"{folderPath}/{item['title']}.txt", "w", encoding="utf-8") as file:
            file.write(
                "".join(re.findall(regex, driver.find_element(By.CSS_SELECTOR, 'pre').text))    
            )
        count += 1
        if count >= 250:
            break
            


In [128]:
if __name__ == '__main__':
    visit()
    getLinks()
    getSublinks()
    
